In [1]:
#file_path="C:/Users/Jonathan/Desktop/Data Analysis and Statistical Methods/dvsa1203.xlsx"


In [2]:
import numpy as np
import pandas as pd
import copy

In [4]:
dict_sheets = pd.read_excel('./dvsa1203.ods', engine='odf', sheet_name=None)
dfs = []

for sheet_name in list(dict_sheets)[1:]:
    df = dict_sheets[sheet_name].copy().iloc[6:].reset_index(drop=True)
    df = df.dropna(axis=1, how="all")
    df.columns = [
        "Location", "Age", "Male Conducted", "Male Passes", "Male Pass rate",
        "Female Conducted", "Female Passes", "Female Pass rate",
        "Total Conducted", "Total Passes", "Total Pass rate"
    ]

    # Forward fill missing locations
    df["Location"] = df["Location"].ffill()

    # Convert Age to numeric and filter by range
    df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
    df = df[df["Age"].between(17, 25)]

    # Add Year column
    df["Year"] = int(sheet_name.split("-")[0])

    # Convert all except Location to numeric
    numeric_cols = df.columns.difference(["Location"])
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

    # Mapping for genders
    gender_mappings = {
        "Male": ["Location", "Year", "Age", "Male Conducted", "Male Passes", "Male Pass rate"],
        "Female": ["Location", "Year", "Age", "Female Conducted", "Female Passes", "Female Pass rate"],
        "Total": ["Location", "Year", "Age", "Total Conducted", "Total Passes", "Total Pass rate"]
    }

    # Create a combined dataframe for all genders
    df_list = []
    for gender, cols in gender_mappings.items():
        sub_df = df[cols].copy()
        sub_df.columns = ["Location", "Year", "Age", "Conducted", "Passes", "Pass rate"]
        sub_df.insert(2, "Gender", gender)
        df_list.append(sub_df)

    df = pd.concat(df_list, ignore_index=True)

    # Convert Pass rate to fraction
    df["Pass rate"] /= 100

    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)


In [5]:
df.head()

,Location,Year,Gender,Age,Conducted,Passes,Pass rate
0,Aberdeen North,2023,Male,17.0,301.0,180.0,0.598007
1,Aberdeen North,2023,Male,18.0,175.0,87.0,0.497143
2,Aberdeen North,2023,Male,19.0,123.0,54.0,0.439024
3,Aberdeen North,2023,Male,20.0,82.0,44.0,0.536585
4,Aberdeen North,2023,Male,21.0,55.0,34.0,0.618182


In [6]:
unique_years1 = df['Year'].unique()

# Display the unique values
print("Unique values in 'Year' column:", unique_years1)

Unique values in 'Year' column: [2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010
 2009 2008 2007]


In [7]:
# Assuming your DataFrame is named `df`

#filtered_df = df[(df['Location'] == 'Camborne') | (df['Location'] == 'Wood Green (London)')]
# filtered_df = df[
#     ((df['Location'] == 'Camborne') | (df['Location'] == 'Wood Green (London)')) &
#     (df['Gender'].isin(['Male', 'Female']))
# ]
desired_locations = ['Camborne', 'Wood Green (London)','Wood Green']
desired_genders = ['Male', 'Female']

# Filtering the DataFrame
filtered_df = df[(df['Location'].isin(desired_locations)) & (df['Gender'].isin(desired_genders))]
# Display the resulting DataFrame
print(filtered_df)

          Location  Year  Gender   Age  Conducted  Passes  Pass rate
590       Camborne  2023    Male  17.0      775.0   361.0   0.465806
591       Camborne  2023    Male  18.0      536.0   213.0   0.397388
592       Camborne  2023    Male  19.0      284.0   120.0   0.422535
593       Camborne  2023    Male  20.0      200.0    78.0   0.390000
594       Camborne  2023    Male  21.0      132.0    57.0   0.431818
...            ...   ...     ...   ...        ...     ...        ...
145482  Wood Green  2007  Female  21.0      250.0   101.0   0.404000
145483  Wood Green  2007  Female  22.0      233.0    90.0   0.386266
145484  Wood Green  2007  Female  23.0      247.0    98.0   0.396761
145485  Wood Green  2007  Female  24.0      186.0    75.0   0.403226
145486  Wood Green  2007  Female  25.0      226.0    82.0   0.362832

[612 rows x 7 columns]


In [8]:
unique_years = filtered_df['Year'].unique()

# Display the unique values
print("Unique values in 'Year' column:", unique_years)

Unique values in 'Year' column: [2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010
 2009 2008 2007]


In [9]:
cleaned_df=copy.deepcopy(filtered_df)

In [ ]:
# Map Gender: Male -> 1, Female -> 0
cleaned_df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

# Map Location: Camborne -> 1, Wood Green -> 0
#cleaned_df['Location'] = df['Location'].map({'Camborne': 1, 'Wood Green (London)': 0})
cleaned_df['Location'] = df['Location'].map({'Camborne': 1, 'Wood Green': 0, 'Wood Green (London)': 0})







In [11]:
# Load your dataset (replace 'your_dataset.csv' with the actual file path) 
# cleaned_df = pd.read_csv('your_dataset.csv')

# 1. Check for missing values
print("Missing values per column:")
print(cleaned_df.isnull().sum())
print("\nRows with missing values:")
print(cleaned_df[cleaned_df.isnull().any(axis=1)])

# 2. Check for duplicate rows
print("\nNumber of duplicate rows:", cleaned_df.duplicated().sum())

# 3. Verify data types
print("\nData types of columns:")
print(cleaned_df.dtypes)

# 4. Check unique values for categorical columns
print("\nUnique values in 'Gender' column (expected: [1, 0]):")
print(cleaned_df['Gender'].unique())
print("\nUnique values in 'Location' column (expected: [1, 0]):")
print(cleaned_df['Location'].unique())

# 5. Check range and distribution of numerical columns
print("\nSummary statistics for numerical columns:")
print(cleaned_df.describe())

# Example check: Ensure 'Age' is in a reasonable range (e.g., 0 to 120)
print("\nRows with invalid 'Age' values (outside 0-120):")
print(cleaned_df[(cleaned_df['Age'] < 0) | (cleaned_df['Age'] > 120)])

# Example check: Ensure 'Pass rate' is between 0 and 1
print("\nRows with invalid 'Pass rate' values (outside 0-1):")
print(cleaned_df[(cleaned_df['Pass rate'] < 0) | (cleaned_df['Pass rate'] > 1)])

# 6. Check the shape of the DataFrame
print("\nShape of the dataset (rows, columns):", cleaned_df.shape)


Missing values per column:
Location     0
Year         0
Gender       0
Age          0
Conducted    0
Passes       0
Pass rate    0
dtype: int64

Rows with missing values:
Empty DataFrame
Columns: [Location, Year, Gender, Age, Conducted, Passes, Pass rate]
Index: []

Number of duplicate rows: 0

Data types of columns:
Location     float64
Year           int64
Gender       float64
Age          float64
Conducted    float64
Passes       float64
Pass rate    float64
dtype: object

Unique values in 'Gender' column (expected: [1, 0]):
[1. 0.]

Unique values in 'Location' column (expected: [1, 0]):
[1. 0.]

Summary statistics for numerical columns:
         Location         Year      Gender         Age   Conducted  \
count  612.000000   612.000000  612.000000  612.000000  612.000000   
mean     0.500000  2015.000000    0.500000   21.000000  199.993464   
std      0.500409     4.902987    0.500409    2.584101  136.454101   
min      0.000000  2007.000000    0.000000   17.000000   18.000000   


In [13]:
cleaned_df.to_excel('preprocessed_data.xlsx',index=False)